# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링

### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?

In [92]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

import pandas as pd
from selenium.webdriver.common.by import By

https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2
https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2
https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2
https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2
https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2

In [164]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소 이동 (TEST)
url = 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2'
driver.get(url)
time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

### User 정보 찾기

In [169]:
# User ID 찾기
user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
user_id = user_id_element.text if user_id_element else None

# 리뷰, 팔로잉, 팔로워 찾기 
user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
user_element = {}
for element in user_popularity:
    em_tag = element.find('em')
    if em_tag:
        key = element.text.replace(em_tag.text, '').strip()
        value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
        user_element[key] = value

print('아이디 :', user_id,user_element)

AttributeError: 'WebElement' object has no attribute 'sub'

### 매장 정보 찾기

In [168]:
# 맨 처음 게시물 클릭
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(0.4)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [96]:
# # 클래스명이 'ptr__children'인 모든 요소를 찾음
# ptr_children_elements = driver.find_elements(By.CLASS_NAME, 'ptr__children')

# # 각 요소의 텍스트를 출력
# for element in ptr_children_elements:
#     print(element.text)


### 리뷰 크롤링

- 아직 안된 부분 : pagedown하기 , tag / 시간 부분, 매장 정보 나누기 


## 이정훈이 여기 해결하는중

In [185]:
# data를 담을 빈 리스트 선언
data_list = [] 

# 리뷰 페이지를 크롤링
ptr_children_elements = driver.find_elements(By.CLASS_NAME, 'ptr__children')

# 페이지에서 각각 클래스 찾기 
for ptr_element in ptr_children_elements:
    restaurant_elements = ptr_element.find_elements(By.CLASS_NAME, '_1QGRWW') 
    category_location_elements = ptr_element.find_elements(By.CLASS_NAME, '_2vBfgu')
    review_text_elements = ptr_element.find_elements(By.CLASS_NAME, 'z0t_8b')
    reactions_elements = ptr_element.find_elements(By.CLASS_NAME, 'COw42b')
    reactions = [span.text for span in driver.find_element(By.CLASS_NAME, '_1oWLID').find_elements(By.CLASS_NAME, 'COw42b')]
    date_info = driver.find_element(By.CLASS_NAME, '_15xwjO').find_element(By.CLASS_NAME, 'hol3Ic').find_element(By.TAG_NAME, 'div')

    # 레스토랑 클래스가 존재하는 경우, 모든 정보를 가져와서 딕셔너리로 저장하기
    for i in range(len(restaurant_elements)) :
        if restaurant_elements:
            restaurant = restaurant_elements[i].text if len(restaurant_elements[i].text) > 0 else '없음'
            category_location = category_location_elements[i].text if len(category_location_elements[i].text) > 0 else '없음'
            # category = category_location[0].text if len(category_location) > 0 else '없음'
            # location = category_location[1].text if len(category_location) > 1 else '없음'
            review_text = review_text_elements[i].text if len(review_text_elements[i].text) > 0 else '없음'

            reactions = reactions_elements[i].text if len(reactions_elements[i].text) > 0 else '없음'
            

            # date_element = date_info.find_element(By.CLASS_NAME, '._3nNYBi') # 여기 수정
            # day = date_element.text if date_element else '없음'
            
            # 딕셔너리 만들고, list로 추가하기
            data_dict = {
                # '아이디' : [user_id],
                # '리뷰' : user_element['리뷰'],
                # '팔로워' : user_element['팔로워'],
                '매장명': restaurant,
                '매장정보': category_location,
                '리뷰 내용': review_text,
                '태그': reactions,
                #'방문일자': day
            }
            data_list.append(data_dict)
    
# 결과 확인
df = pd.DataFrame(data_list)
df

,매장명,매장정보,리뷰 내용,태그
0,파스텔드나따 압구정,베이커리서울특별시 강남구 신사동,일요일 오후에 방문했는데 직원분 너무 친절하고 겸손하셔서 놀라고 갑니다! 에그타르트...,특별한 메뉴가 있어요
1,단데농,"카페,디저트서울특별시 서대문구 창천동",결혼식 가는길 버스 대기시간이 길어서 들렸는데 사장님 너무 예쁘시고 런더포그 너무 ...,디저트가 맛있어요
2,스윗센츠,"카페,디저트서울특별시 서초구 양재동",양재청 로스터리 카페 스윗센츠! 직접 로스팅을 해서 커피맛이 신선하구요! 시그니처 ...,친절해요
3,덴키카츠,돈가스서울특별시 강남구 삼성동,돈가스도 카레우동도 다\n맛있어요!!!,커피가 맛있어요
4,우드왁싱 선릉점,"왁싱,제모서울특별시 강남구 대치동",샵이 깨끗하고 쾌적해요!\n시술도 정말 꼼꼼하게 잘 해주시고 마무리 케어도 훌륭해요...,+4\n개의 리뷰 더 있습니다
5,야익스커피,카페대전광역시 서구 월평동,커피맛 깔끔하고 파운드 케이크도 너무 맛있어요!\n분위기가 조용해서 작업하기도 좋고...,커피가 맛있어요
6,태희갤러리카페,"카페,디저트대전광역시 유성구 장대동",조용하고 좋아요!,+4\n개의 리뷰 더 있습니다
7,창억떡집 대전점,"떡,한과대전광역시 유성구 장대동",대전 놀러왔다가 창억떡집 보고 신기해서 들어갔다가 밤통팥찰떡 샀어요! 찰떡 사면 재...,음식이 맛있어요
8,따봉하우스,떡볶이서울특별시 동대문구 이문동,친구가 20년 다닌 맛집이라며 데려가줬는데 예스러움에 반하고 맛있고 친절+유쾌하셔서...,+3\n개의 리뷰 더 있습니다
9,보자기두부 서초점,두부제조서울특별시 서초구 서초동,이름도 너무 예쁜 보자기두부! 매장에서 직접 만드는 두부라 안심하고 먹을 수 있어요...,친절해요


In [206]:
# data를 담을 빈 리스트 선언
data_list = [] 

# 리뷰 페이지를 크롤링
ptr_children_elements = driver.find_elements(By.CLASS_NAME, 'ptr__children')

# 페이지에서 각각 클래스 찾기 
for ptr_element in ptr_children_elements:
    restaurant_elements = ptr_element.find_elements(By.CLASS_NAME, '_1QGRWW') 
    category_location_elements = ptr_element.find_elements(By.CLASS_NAME, '_2vBfgu')
    review_text_elements = ptr_element.find_elements(By.CLASS_NAME, 'z0t_8b')
    reactions_elements = ptr_element.find_elements(By.CLASS_NAME, '_1oWLID')
    for x in reactions_elements:
        print(x.text)
    reactions=[x.text for x in driver.find_element(By.CSS_SELECTOR,"._1oWLID").find_elements(By.CSS_SELECTOR,'.COw42b')]
    date_info = driver.find_element(By.CLASS_NAME, '_15xwjO').find_element(By.CLASS_NAME, 'hol3Ic').find_element(By.TAG_NAME, 'div')

    # 레스토랑 클래스가 존재하는 경우, 모든 정보를 가져와서 딕셔너리로 저장하기
    for i in range(len(restaurant_elements)) :
        if restaurant_elements:
            restaurant = restaurant_elements[i].text if len(restaurant_elements[i].text) > 0 else '없음'
            category_location = category_location_elements[i].text if len(category_location_elements[i].text) > 0 else '없음'
            # category = category_location[0].text if len(category_location) > 0 else '없음'
            # location = category_location[1].text if len(category_location) > 1 else '없음'
            review_text = review_text_elements[i].text if len(review_text_elements[i].text) > 0 else '없음'

            #reactions = reactions_elements[i].text if len(reactions_elements[i].text) > 0 else '없음'

            # date_element = date_info.find_element(By.CLASS_NAME, '._3nNYBi') # 여기 수정
            # day = date_element.text if date_element else '없음'
            
            # 딕셔너리 만들고, list로 추가하기
            data_dict = {
                # '아이디' : [user_id],
                # '리뷰' : user_element['리뷰'],
                # '팔로워' : user_element['팔로워'],
                '매장명': restaurant,
                '매장정보': category_location,
                '리뷰 내용': review_text,
                '태그': reactions,
                #'방문일자': day
            }
            data_list.append(data_dict)
    
# 결과 확인
df = pd.DataFrame(data_list)
df

특별한 메뉴가 있어요디저트가 맛있어요친절해요
커피가 맛있어요+4
개의 리뷰 더 있습니다
커피가 맛있어요+4
개의 리뷰 더 있습니다
음식이 맛있어요+3
개의 리뷰 더 있습니다
친절해요+3
개의 리뷰 더 있습니다
커피가 맛있어요+3
개의 리뷰 더 있습니다
커피가 맛있어요+4
개의 리뷰 더 있습니다
재료가 신선해요+1
개의 리뷰 더 있습니다
음식이 맛있어요+2
개의 리뷰 더 있습니다
감각적이에요+4
개의 리뷰 더 있습니다


,매장명,매장정보,리뷰 내용,태그
0,파스텔드나따 압구정,베이커리서울특별시 강남구 신사동,일요일 오후에 방문했는데 직원분 너무 친절하고 겸손하셔서 놀라고 갑니다! 에그타르트...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
1,단데농,"카페,디저트",결혼식 가는길 버스 대기시간이 길어서 들렸는데 사장님 너무 예쁘시고 런더포그 너무 ...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
2,스윗센츠,"카페,디저트",양재청 로스터리 카페 스윗센츠! 직접 로스팅을 해서 커피맛이 신선하구요! 시그니처 ...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
3,덴키카츠,돈가스서울특별시 강남구 삼성동,돈가스도 카레우동도 다\n맛있어요!!!,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
4,우드왁싱 선릉점,"왁싱,제모서울특별시 강남구 대치동",샵이 깨끗하고 쾌적해요!\n시술도 정말 꼼꼼하게 잘 해주시고 마무리 케어도 훌륭해요...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
5,야익스커피,카페대전광역시 서구 월평동,커피맛 깔끔하고 파운드 케이크도 너무 맛있어요!\n분위기가 조용해서 작업하기도 좋고...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
6,태희갤러리카페,"카페,디저트",조용하고 좋아요!,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
7,창억떡집 대전점,"떡,한과대전광역시 유성구 장대동",대전 놀러왔다가 창억떡집 보고 신기해서 들어갔다가 밤통팥찰떡 샀어요! 찰떡 사면 재...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
8,따봉하우스,떡볶이서울특별시 동대문구 이문동,친구가 20년 다닌 맛집이라며 데려가줬는데 예스러움에 반하고 맛있고 친절+유쾌하셔서...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"
9,보자기두부 서초점,두부제조서울특별시 서초구 서초동,이름도 너무 예쁜 보자기두부! 매장에서 직접 만드는 두부라 안심하고 먹을 수 있어요...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]"


In [192]:
df.head(0)["태그"]

Series([], Name: 태그, dtype: object)

### 매장 리뷰 크롤링

In [41]:
# 매장명
restaurant = driver.find_element(By.CLASS_NAME, '_1QGRWW').text
restaurant = restaurant if len(restaurant) > 0 else '없음'

# 주소 
category_location = driver.find_element(By.CLASS_NAME, '_2vBfgu').find_elements(By.TAG_NAME, 'span')
category = category_location[0].text if len(category_location) > 0 else '없음'
location = category_location[1].text if len(category_location) > 1 else '없음'

# 리뷰
review_text = driver.find_element(By.CLASS_NAME, 'z0t_8b').find_element(By.CLASS_NAME, '_3-ITu7').text
review_text = review_text if len(review_text) > 0 else '없음'

# tag 
reactions = [span.text for span in driver.find_element(By.CLASS_NAME, '_1oWLID').find_elements(By.CLASS_NAME, 'COw42b')]
reactions = reactions if len(reactions) > 0 else '없음'

# 방문일자 
date_info = driver.find_element(By.CLASS_NAME, '_15xwjO').find_element(By.CLASS_NAME, 'hol3Ic').find_element(By.TAG_NAME, 'div')
date_element = date_info.find_element(By.XPATH, './span[@class="_3nNYBi"]/time')
day = date_element.text if date_element else '없음'

# Create a DataFrame
data = {
    # '아이디' : [user_id],
    # '리뷰' : user_element['리뷰'],
    # '팔로워' : user_element['팔로워'],
    '매장명': [restaurant],
    '카테고리': [category],
    '주소': [location],
    '리뷰 내용': [review_text],
    '태그': [reactions],
    '방문일자': [day]
}

User_df = pd.DataFrame(data)